In [131]:
import pandas as pd
pd.set_option("display.max_rows", None)

# Lista para armazenar os dados da tabela
nuppa = pd.read_csv('NUPPA_CSV.csv')
nuppa.head()

,Arquivo,Produtos,Valor Comum,Valor Máximo,Valor Mediana,Valor Médio,Valor Mínimo,Semana,Dia,Mês,Ano,Data
0,01_22,Abobrinha Italiana,90.0,120.0,90.0,94.29,80.0,1,24,3,2022,2022-03-24 00:00:00
1,02_22,Abobrinha Italiana,70.0,100.0,70.0,72.78,60.0,2,31,3,2022,2022-03-31 00:00:00
2,03_22,Abobrinha Italiana,70.0,100.0,75.0,77.50,70.0,3,7,4,2022,2022-04-07 00:00:00
3,04_22,Abobrinha Italiana,70.0,80.0,70.0,68.13,50.0,4,14,4,2022,2022-04-14 00:00:00
4,05_22,Abobrinha Italiana,50.0,90.0,50.0,52.00,35.0,5,28,4,2022,2022-04-28 00:00:00


In [132]:
inmet = pd.read_csv('INMET_SP_2022_2023.csv')
inmet.head()

,Data,Média Precipitação Total (mm),Média Pressão Atmosférica (mB),Média Temperatura do Ar (°C),Média Umidade Relativa do Ar (%),Média Vento Velocidade (m/s)
0,2022-01-01 00:00:00,0.025000,921.800000,21.237500,"90,25",1.083333
1,2022-01-02 00:00:00,0.000000,923.770833,22.966667,"82,04",1.275000
2,2022-01-03 00:00:00,0.041667,923.562500,23.079167,"84,50",1.562500
3,2022-01-04 00:00:00,0.533333,921.195833,22.395833,"93,33",1.270833
4,2022-01-05 00:00:00,0.500000,919.570833,23.279167,"87,21",1.933333


In [133]:
nuppa_inmet = pd.merge(nuppa,inmet)

nuppa_inmet['Média Umidade Relativa do Ar (%)'] = nuppa_inmet['Média Umidade Relativa do Ar (%)'].str.replace(',','.').astype(float)

nuppa_inmet.dtypes

Arquivo                              object
Produtos                             object
Valor Comum                         float64
Valor Máximo                        float64
Valor Mediana                       float64
Valor Médio                         float64
Valor Mínimo                        float64
Semana                                int64
Dia                                   int64
Mês                                   int64
Ano                                   int64
Data                                 object
Média Precipitação Total (mm)       float64
Média Pressão Atmosférica (mB)      float64
Média Temperatura do Ar (°C)        float64
Média Umidade Relativa do Ar (%)    float64
Média Vento Velocidade (m/s)        float64
dtype: object

In [137]:
nuppa_inmet['Média Precipitação Total (mm)'] = nuppa_inmet['Média Precipitação Total (mm)'].round(4)
nuppa_inmet['Média Pressão Atmosférica (mB)'] = nuppa_inmet['Média Pressão Atmosférica (mB)'].round(3)
nuppa_inmet['Média Temperatura do Ar (°C)'] = nuppa_inmet['Média Temperatura do Ar (°C)'].round(2)
nuppa_inmet['Média Umidade Relativa do Ar (%)'] = nuppa_inmet['Média Umidade Relativa do Ar (%)'].round(2)
nuppa_inmet['Média Vento Velocidade (m/s)'] = nuppa_inmet['Média Vento Velocidade (m/s)'].round(2)

nuppa_inmet['Média Umidade Relativa do Ar (%)'] = nuppa_inmet['Média Umidade Relativa do Ar (%)'].astype(str) + '%'

nuppa_inmet.head()

,Arquivo,Produtos,Valor Comum,Valor Máximo,Valor Mediana,Valor Médio,Valor Mínimo,Semana,Dia,Mês,Ano,Data,Média Precipitação Total (mm),Média Pressão Atmosférica (mB),Média Temperatura do Ar (°C),Média Umidade Relativa do Ar (%),Média Vento Velocidade (m/s)
0,01_22,Abobrinha Italiana,90.0,120.0,90.0,94.29,80.0,1,24,3,2022,2022-03-24 00:00:00,0.0,925.846,23.93,71.25%,1.94
1,01_22,Acelga,35.0,60.0,50.0,42.50,35.0,1,24,3,2022,2022-03-24 00:00:00,0.0,925.846,23.93,71.25%,1.94
2,01_22,Agrião,60.0,60.0,60.0,60.00,60.0,1,24,3,2022,2022-03-24 00:00:00,0.0,925.846,23.93,71.25%,1.94
3,01_22,Alface Americana,36.0,42.0,39.0,39.00,36.0,1,24,3,2022,2022-03-24 00:00:00,0.0,925.846,23.93,71.25%,1.94
4,01_22,Alface Crespa,20.0,30.0,25.0,25.00,20.0,1,24,3,2022,2022-03-24 00:00:00,0.0,925.846,23.93,71.25%,1.94


In [138]:
inmet = nuppa_inmet.loc[:, 'Data':'Média Vento Velocidade (m/s)']
inmet.to_csv('INMET_SP_2022_2023_filtered.csv', index = 0)